In [1]:
# Import necessary libraries
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# Data preparation
img_size = 224  # Image size to resize
categories = ['Lonely', 'Not_Lonely']
data_dir = 'F:/lonely_non'

In [3]:
def load_data(data_dir):
    data = []
    labels = []
    
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                resized_array = cv2.resize(img_array, (img_size, img_size))
                data.append(resized_array)
                labels.append(class_num)
            except Exception as e:
                pass
    
    return np.array(data), np.array(labels)


In [4]:
# Load data
X, y = load_data(data_dir)
y = to_categorical(y, num_classes=2)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Normalize the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [7]:
# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False  # Freeze the base model layers

In [8]:
# Create a new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')  # 2 classes: 'Lonely' and 'Not_Lonely'
])

In [9]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model training
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32,
                    callbacks=[checkpoint, early_stop])

# Load the best saved model
model.load_weights('best_model.keras')

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 503ms/step - accuracy: 0.7414 - loss: 0.5313 - val_accuracy: 0.9384 - val_loss: 0.2031
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 13s 486ms/step - accuracy: 0.9501 - loss: 0.1493 - val_accuracy: 0.9336 - val_loss: 0.1633
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 561ms/step - accuracy: 0.9562 - loss: 0.1181 - val_accuracy: 0.9431 - val_loss: 0.1442
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 516ms/step - accuracy: 0.9771 - loss: 0.0831 - val_accuracy: 0.9479 - val_loss: 0.1381
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 13s 492ms/step - accuracy: 0.9864 - loss: 0.0757 - val_accuracy: 0.9479 - val_loss: 0.1440
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 509ms/step - accuracy: 0.9909 - loss: 0.0556 - val_accuracy: 0.9526 - val_loss: 0.1282
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 13s 494ms/step - accuracy: 0.9956 - loss: 0.0456 - val_accuracy: 0.9526 - val_loss: 0.1235
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 509ms/step - accuracy: 0.9975 - loss: 0.0397 - val_accu

In [10]:
# Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 481ms/step


In [11]:
accuracy = accuracy_score(y_true, y_pred_classes)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.96


In [12]:
print(classification_report(y_true, y_pred_classes, target_names=categories))

              precision    recall  f1-score   support

      Lonely       0.95      0.97      0.96       119
  Not_Lonely       0.97      0.93      0.95        92

    accuracy                           0.96       211
   macro avg       0.96      0.95      0.96       211
weighted avg       0.96      0.96      0.96       211



In [13]:
model.save('stj_alone.keras')


In [2]:
import cv2
import numpy as np
import tensorflow as tf
from collections import deque

# Load the trained model
model = tf.keras.models.load_model('C:/Users/matha/Documents/SIH/GITHUB/Women_Safety_Analytics/Models/Lonely_Non-Lonely.keras')

# Define image size to match the model input size
img_size = 224
categories = ['Lonely', 'Not_Lonely']

# Function to preprocess the frame for the model
def preprocess_frame(frame):
    img_resized = cv2.resize(frame, (img_size, img_size))
    img_normalized = img_resized.astype('float32') / 255.0  # Normalize pixel values
    img_input = np.expand_dims(img_normalized, axis=0)  # Add batch dimension
    return img_input

# Reduce frame rate or skip frames to ease resource usage
frame_skip = 5  # Process every 5th frame
frame_count = 0

# Smooth predictions over the last few frames to reduce flickering
prediction_window = deque(maxlen=10)  # A deque to hold the last 10 predictions

# Open webcam for real-time prediction
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Initialize the label variable
label = 'Processing...'

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    frame_count += 1

    # Process every 5th frame to reduce load
    if frame_count % frame_skip == 0:
        # Preprocess the frame
        img_input = preprocess_frame(frame)

        # Predict using the model
        prediction = model.predict(img_input)

        # Determine predicted class
        predicted_class = np.argmax(prediction)
        prediction_window.append(predicted_class)  # Store the predicted class

        # Get the most common prediction in the window
        stable_prediction = max(set(prediction_window), key=prediction_window.count)

        # Get the label for the stable predicted class
        label = categories[stable_prediction]

    # Display prediction label on the frame
    cv2.putText(frame, f'{label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Real-Time Detection', frame)

    # Add a delay to update the GUI and check for 'q' key to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━